# Simple Example

This notebook demonstrates how to model a simple energy system with **flixopt**, including:

- Multiple energy carriers (electricity, heat, gas)
- A Combined Heat and Power (CHP) unit
- Thermal storage
- Effects for costs and CO2 emissions

This example provides a solid foundation for understanding flixopt's core concepts.

## Setup

In [ ]:
import numpy as np
import pandas as pd

import flixopt as fx

# Configure for notebook environment
fx.CONFIG.notebook()

## Create Time Series Data

First, we define the input data for our model:
- **Heat demand**: The thermal load profile over time (in kW)
- **Power prices**: Electricity prices for feed-in revenue (in €/kWh)

In [ ]:
# Heat demand profile (kW) - varies throughout the day
heat_demand_per_h = np.array([30, 0, 90, 110, 110, 20, 20, 20, 20])

# Electricity prices (€/kWh) - constant for simplicity
power_prices = 1 / 1000 * np.array([80, 80, 80, 80, 80, 80, 80, 80, 80])

# Create datetime index
timesteps = pd.date_range('2020-01-01', periods=len(heat_demand_per_h), freq='h')

print(f'Optimization horizon: {timesteps[0]} to {timesteps[-1]}')
print(f'Number of timesteps: {len(timesteps)}')

## Create FlowSystem

The `FlowSystem` is the main container that holds all components and manages the optimization.

In [ ]:
flow_system = fx.FlowSystem(timesteps=timesteps)

## Define Energy Buses

**Buses** are nodes where energy carriers are balanced. The fundamental constraint is:

$$\sum \text{inputs} = \sum \text{outputs}$$

We use **carriers** to automatically assign colors in plots:
- `electricity` → yellow
- `heat` → red  
- `gas` → blue

In [ ]:
flow_system.add_elements(
    fx.Bus(label='Strom', carrier='electricity'),
    fx.Bus(label='Fernwärme', carrier='heat'),
    fx.Bus(label='Gas', carrier='gas'),
)

## Define Effects

**Effects** are quantities that can be:
- **Tracked** - accumulated over the optimization horizon
- **Constrained** - limited by minimum/maximum values
- **Optimized** - used as the objective function

Here we define:
1. **Costs** - The objective to minimize (with CO2 price linkage)
2. **CO2** - Emissions with an hourly limit

In [ ]:
# Cost effect - used as the optimization objective
costs = fx.Effect(
    label='costs',
    unit='€',
    description='Kosten',
    is_standard=True,  # Costs are automatically assigned if no effect is specified
    is_objective=True,  # Minimize this effect
    share_from_temporal={'CO2': 0.2},  # Add 0.2 €/kg CO2 as carbon price
)

# CO2 emissions effect with hourly constraint
CO2 = fx.Effect(
    label='CO2',
    unit='kg',
    description='CO2_e-Emissionen',
    maximum_per_hour=1000,  # Regulatory limit: max 1000 kg CO2/hour
)

## Define Components

Now we create the actual energy system components:

### Gas Boiler

A simple converter that transforms gas into heat:

In [ ]:
boiler = fx.linear_converters.Boiler(
    label='Boiler',
    thermal_efficiency=0.5,  # 50% efficiency
    thermal_flow=fx.Flow(
        label='Q_th',
        bus='Fernwärme',
        size=50,  # Maximum capacity: 50 kW
        relative_minimum=0.1,  # Minimum part load: 10%
        relative_maximum=1,  # Maximum part load: 100%
    ),
    fuel_flow=fx.Flow(label='Q_fu', bus='Gas'),
)

### Combined Heat and Power (CHP)

A CHP unit generates both electricity and heat from fuel with fixed efficiency ratios:

In [ ]:
chp = fx.linear_converters.CHP(
    label='CHP',
    thermal_efficiency=0.5,  # 50% thermal efficiency
    electrical_efficiency=0.4,  # 40% electrical efficiency
    electrical_flow=fx.Flow('P_el', bus='Strom', size=60, relative_minimum=5 / 60),
    thermal_flow=fx.Flow('Q_th', bus='Fernwärme'),
    fuel_flow=fx.Flow('Q_fu', bus='Gas'),
)

### Thermal Storage

Storage allows temporal decoupling of heat production and demand:

In [ ]:
storage = fx.Storage(
    label='Storage',
    charging=fx.Flow('Q_th_load', bus='Fernwärme', size=1000),
    discharging=fx.Flow('Q_th_unload', bus='Fernwärme', size=1000),
    capacity_in_flow_hours=fx.InvestParameters(
        effects_of_investment=20,  # Investment cost: 20 €
        fixed_size=30,  # Fixed capacity: 30 kWh
        mandatory=True,  # Must be built
    ),
    initial_charge_state=0,  # Start empty
    relative_maximum_charge_state=1 / 100 * np.array([80, 70, 80, 80, 80, 80, 80, 80, 80]),
    relative_maximum_final_charge_state=0.8,
    eta_charge=0.9,  # 90% charging efficiency
    eta_discharge=1,  # 100% discharging efficiency
    relative_loss_per_hour=0.08,  # 8% loss per hour
    prevent_simultaneous_charge_and_discharge=True,
)

### Sinks and Sources

Complete the system with demand (sink) and supply (source) components:

In [ ]:
# Heat demand - fixed profile
heat_sink = fx.Sink(
    label='Heat Demand',
    inputs=[fx.Flow(label='Q_th_Last', bus='Fernwärme', size=1, fixed_relative_profile=heat_demand_per_h)],
)

# Gas supply with costs and emissions
gas_source = fx.Source(
    label='Gastarif',
    outputs=[
        fx.Flow(
            label='Q_Gas',
            bus='Gas',
            size=1000,
            effects_per_flow_hour={
                costs.label: 0.04,  # 0.04 €/kWh
                CO2.label: 0.3,  # 0.3 kg CO2/kWh
            },
        )
    ],
)

# Electricity feed-in (revenue)
power_sink = fx.Sink(
    label='Einspeisung',
    inputs=[
        fx.Flow(
            label='P_el',
            bus='Strom',
            effects_per_flow_hour=-1 * power_prices,  # Negative = revenue
        )
    ],
)

## Build the FlowSystem

Add all components and effects to the flow system:

In [ ]:
flow_system.add_elements(costs, CO2, boiler, storage, chp, heat_sink, gas_source, power_sink)

## Visualize the System Topology

Before optimizing, let's visualize the system structure to verify it's correct:

In [ ]:
flow_system.topology.plot('topology.html', show=False)

## Run the Optimization

Now we solve the optimization problem using the HiGHS solver:

In [ ]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0, time_limit_seconds=30))

## Analyze Results

### Heat Balance

Let's see how the heat demand is met by different components:

In [ ]:
flow_system.statistics.plot.balance('Fernwärme')

### Storage Behavior

Visualize how the storage charges and discharges:

In [ ]:
flow_system.statistics.plot.balance('Storage')

### Heatmap Visualizations

Heatmaps are useful for visualizing patterns over time:

In [ ]:
flow_system.statistics.plot.heatmap('CHP(Q_th)')

In [ ]:
flow_system.statistics.plot.heatmap('Storage')

### Access Raw Data

The results are available as xarray Datasets for further analysis:

In [ ]:
print('Flow rates:')
flow_system.statistics.flow_rates

In [ ]:
print('Charge states:')
flow_system.statistics.charge_states

### Duration Curve

Duration curves show the sorted values, useful for sizing analysis:

In [ ]:
flow_system.statistics.plot.duration_curve('Boiler(Q_th)')

### Effects Summary

View the temporal effects (costs, emissions) over time:

In [ ]:
flow_system.statistics.temporal_effects

## Summary

This example demonstrated:

- Creating a multi-carrier energy system (electricity, heat, gas)
- Using pre-built components (`Boiler`, `CHP`, `Storage`)
- Defining costs and emissions as effects
- Visualizing system topology and results
- Accessing results as xarray Datasets

For more advanced features, see the complex example and optimization modes notebooks!